In [1]:
# notebook to illustrate co-occurrence graphs

In [2]:
# following only used for development, reloads the modules with any code changes
%load_ext autoreload
%autoreload 2

# inline matplotlib charts
%matplotlib inline

In [3]:
# import our text mining toolkit
import text_mining_toolkit as tmt

In [4]:
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/simple_test/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/recipes/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(directory_of_files="data_sets/mystery_corpus_01/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/iraq_inquiry/txt/", text_filename_pattern="the-report*.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/clinton_emails/txt/", text_filename_pattern="C0*.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/shakespeare_macbeth/txt/", text_filename_pattern="macbeth_act_0?_scene_0?.txt")
cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/hillsborough/txt/", text_filename_pattern="*.txt")

content_directory =  data_sets/hillsborough/txt/
text_filename_pattern =  *.txt
self.documents populated =  19217


In [5]:
# load previously calculated relevance scores
words_by_relevance = tmt.index_relevance.get_words_by_relevance(cr.content_directory)
# top most relevant
top_words_by_relevance = words_by_relevance[:50]

In [6]:
top_words_by_relevance[:10]

,relevance
police,83.451991
raised,79.235280
ground,66.290401
actions,65.596721
document,63.932343
action,61.090630
material,57.589417
number,51.046630
indexer,50.493850
instructions,48.442706


In [7]:
# a set is more efficient when checking for membership
top_words_by_relevance_set = set(top_words_by_relevance.index.tolist())
top_words_by_relevance_set

{'action',
 'actions',
 'april',
 'authority',
 'chief',
 'could',
 'crowd',
 'disaster',
 'document',
 'evidence',
 'football',
 'further',
 'ground',
 'house',
 'indexed',
 'indexer',
 'indicated',
 'inquiry',
 'instructions',
 'justice',
 'letter',
 'liverpool',
 'london',
 'material',
 'number',
 'office',
 'officer',
 'officers',
 'other',
 'people',
 'pitch',
 'police',
 'raised',
 'reference',
 'report',
 'safety',
 'secretary',
 'sheffield',
 'should',
 'south',
 'stand',
 'statement',
 'supporters',
 'taylor',
 'telephone',
 'there',
 'which',
 'witness',
 'would',
 'yorkshire'}

In [8]:
# first clear index
tmt.index_cooccurrence.delete_matrix(cr.content_directory)

# for all documents in corpus
for document_name in cr.get_documents():
    print("processing ", document_name)

    # get document text
    document_text = cr.get_text_by_document(document_name)

    # simplify whitespace (remove newlines)
    b = tmt.text_processing.simplify_whitespace(document_text)

    # only keep alphanumeric characters, removes punctuation
    c = tmt.text_processing.keep_only_alphanumeric(b)

    # make lowercase
    d = tmt.text_processing.to_lowercase(c)

    # split into words list
    dl = tmt.text_processing.split_text_into_words(d)
    # build n-grams
    #gl = tmt.word_processing.build_ngrams_from_words(dl,2)

    # remove stop words
    el = tmt.word_processing.remove_stop_words(dl, "./stopwords/minimal-stop.txt")
    
    # only keep words with min length 5
    fl = tmt.word_processing.keep_words_min_length(el,5)
    
    # remove all except those in the top most relevant list
    gl = [word for word in fl if word in top_words_by_relevance_set]
    
    # update index
    tmt.index_cooccurrence.create_cooccurrence_matrix_for_document(cr.content_directory, document_name, gl, window=4)
    pass


processing  AGO000000010001.txt
processing  AGO000000020001.txt
processing  AGO000000030001.txt
processing  AGO000000040001.txt
processing  AGO000000050001.txt
processing  AGO000000060001.txt
processing  AGO000000070001.txt
processing  AGO000000080001.txt
processing  AGO000000090001.txt
processing  AGO000000100001.txt
processing  AGO000000110001.txt
processing  AGO000000120001.txt
processing  AGO000000130001.txt
processing  AGO000000140001.txt
processing  AGO000000150001.txt
processing  AGO000000160001.txt
processing  AGO000000180001.txt
processing  AGO000000200001.txt
processing  AGO000000210001.txt
processing  AGO000000220001.txt
processing  AGO000000230001.txt
processing  AGO000000240001.txt
processing  AGO000000250001.txt
processing  AGO000000260001.txt
processing  AGO000000270001.txt
processing  AGO000000280001.txt
processing  AGO000000290001.txt
processing  AGO000000300001.txt
processing  AGO000000310001.txt
processing  AGO000000320001.txt
processing  AGO000000330001.txt
processi

processing  CMS000000780001.txt
processing  CMS000000790001.txt
processing  CMS000000800001.txt
processing  CMS000000840001.txt
processing  CMS000000850001.txt
processing  CMS000000860001.txt
processing  CMS000000870001.txt
processing  CMS000000880001.txt
processing  CMS000000910001.txt
processing  CMS000000920001.txt
processing  CMS000000930001.txt
processing  CMS000000940001.txt
processing  CMS000000950001.txt
processing  CMS000000960001.txt
processing  CMS000000980001.txt
processing  CMS000000990001.txt
processing  CMS000001000001.txt
processing  CMS000001010001.txt
processing  CMS000001040001.txt
processing  CMS000001050001.txt
processing  CMS000001060001.txt
processing  CMS000001070001.txt
processing  CMS000001080001.txt
processing  CMS000001090001.txt
processing  CMS000001100001.txt
processing  CMS000001110001.txt
processing  CMS000001140001.txt
processing  CMS000001150001.txt
processing  CMS000001160001.txt
processing  CMS000001170001.txt
processing  CMS000001180001.txt
processi

processing  CMS000004060001.txt
processing  CMS000004070001.txt
processing  CMS000004090001.txt
processing  CMS000004110001.txt
processing  CMS000004130001.txt
processing  CMS000004140001.txt
processing  CMS000004150001.txt
processing  CMS000004160001.txt
processing  CMS000004180001.txt
processing  CMS000004190001.txt
processing  CMS000004200001.txt
processing  CMS000004210001.txt
processing  CMS000004220001.txt
processing  CMS000004230001.txt
processing  CMS000004240001.txt
processing  CMS000004250001.txt
processing  CMS000004260001.txt
processing  CMS000004270001.txt
processing  CMS000004280001.txt
processing  CMS000004290001.txt
processing  CMS000004300001.txt
processing  CMS000004310001.txt
processing  CMS000004320001.txt
processing  CMS000004330001.txt
processing  CMS000004340001.txt
processing  CMS000004360001.txt
processing  CMS000004370001.txt
processing  CMS000004380001.txt
processing  CMS000004400001.txt
processing  CMS000004410001.txt
processing  CMS000004420001.txt
processi

processing  CMS000007080001.txt
processing  CMS000007090001.txt
processing  CMS000007100001.txt
processing  CMS000007120001.txt
processing  CMS000007130001.txt
processing  CMS000007140001.txt
processing  CMS000007150001.txt
processing  CMS000007160001.txt
processing  CMS000007170001.txt
processing  CMS000007180001.txt
processing  CMS000007190001.txt
processing  CMS000007200001.txt
processing  CMS000007210001.txt
processing  CMS000007220001.txt
processing  CMS000007240001.txt
processing  CMS000007250001.txt
processing  CMS000007260001.txt
processing  CMS000007270001.txt
processing  CMS000007290001.txt
processing  CMS000007310001.txt
processing  CMS000007320001.txt
processing  CMS000007330001.txt
processing  CMS000007340001.txt
processing  CMS000007350001.txt
processing  CMS000007360001.txt
processing  CMS000007380001.txt
processing  CMS000007400001.txt
processing  CMS000007410001.txt
processing  CMS000007430001.txt
processing  CMS000007440001.txt
processing  CMS000007550001.txt
processi

processing  CMS000010290001.txt
processing  CMS000010300001.txt
processing  CMS000010310001.txt
processing  CMS000010320001.txt
processing  CMS000010330001.txt
processing  CMS000010340001.txt
processing  CMS000010350001.txt
processing  CMS000010360001.txt
processing  CMS000010370001.txt
processing  CMS000010380001.txt
processing  CMS000010390001.txt
processing  CMS000010400001.txt
processing  CMS000010410001.txt
processing  CMS000010420001.txt
processing  CMS000010430001.txt
processing  CMS000010440001.txt
processing  CMS000010450001.txt
processing  CMS000010460001.txt
processing  CMS000010470001.txt
processing  CMS000010480001.txt
processing  CMS000010500001.txt
processing  CMS000010510001.txt
processing  CMS000010520001.txt
processing  CMS000010530001.txt
processing  CMS000010540001.txt
processing  CMS000010550001.txt
processing  CMS000010560001.txt
processing  CMS000010570001.txt
processing  CMS000010580001.txt
processing  CMS000010590001.txt
processing  CMS000010600001.txt
processi

...

processing  SYP000063360001.txt
processing  SYP000063370001.txt
processing  SYP000063380001.txt
processing  SYP000063390001.txt
processing  SYP000063400001.txt
processing  SYP000063410001.txt
processing  SYP000063420001.txt
processing  SYP000063430001.txt
processing  SYP000063440001.txt
processing  SYP000063450001.txt
processing  SYP000063460001.txt
processing  SYP000063470001.txt
processing  SYP000063480001.txt
processing  SYP000063490001.txt
processing  SYP000063510001.txt
processing  SYP000063520001.txt
processing  SYP000063530001.txt
processing  SYP000063540001.txt
processing  SYP000063550001.txt
processing  SYP000063560001.txt
processing  SYP000063570001.txt
processing  SYP000063580001.txt
processing  SYP000063590001.txt
processing  SYP000063600001.txt
processing  SYP000063610001.txt
processing  SYP000063620001.txt
processing  SYP000063630001.txt
processing  SYP000063640001.txt
processing  SYP000063650001.txt
processing  SYP000063660001.txt
processing  SYP000063670001.txt
processi

processing  SYP000066410001.txt
processing  SYP000066420001.txt
processing  SYP000066430001.txt
processing  SYP000066440001.txt
processing  SYP000066450001.txt
processing  SYP000066460001.txt
processing  SYP000066470001.txt
processing  SYP000066490001.txt
processing  SYP000066510001.txt
processing  SYP000066530001.txt
processing  SYP000066540001.txt
processing  SYP000066550001.txt
processing  SYP000066570001.txt
processing  SYP000066580001.txt
processing  SYP000066590001.txt
processing  SYP000066600001.txt
processing  SYP000066610001.txt
processing  SYP000066620001.txt
processing  SYP000066630001.txt
processing  SYP000066640001.txt
processing  SYP000066650001.txt
processing  SYP000066670001.txt
processing  SYP000066680001.txt
processing  SYP000066690001.txt
processing  SYP000066700001.txt
processing  SYP000066710001.txt
processing  SYP000066720001.txt
processing  SYP000066730001.txt
processing  SYP000066740001.txt
processing  SYP000066750001.txt
processing  SYP000066760001.txt
processi

processing  SYP000069740001.txt
processing  SYP000069770001.txt
processing  SYP000069780001.txt
processing  SYP000069790001.txt
processing  SYP000069800001.txt
processing  SYP000069810001.txt
processing  SYP000069820001.txt
processing  SYP000069830001.txt
processing  SYP000069840001.txt
processing  SYP000069850001.txt
processing  SYP000069860001.txt
processing  SYP000069890001.txt
processing  SYP000069900001.txt
processing  SYP000069910001.txt
processing  SYP000069970001.txt
processing  SYP000069990001.txt
processing  SYP000070000001.txt
processing  SYP000070010001.txt
processing  SYP000070020001.txt
processing  SYP000070030001.txt
processing  SYP000070040001.txt
processing  SYP000070050001.txt
processing  SYP000070060001.txt
processing  SYP000070070001.txt
processing  SYP000070080001.txt
processing  SYP000070090001.txt
processing  SYP000070130001.txt
processing  SYP000070140001.txt
processing  SYP000070150001.txt
processing  SYP000070160001.txt
processing  SYP000070180001.txt
processi

processing  SYP000072830001.txt
processing  SYP000072840001.txt
processing  SYP000072850001.txt
processing  SYP000072860001.txt
processing  SYP000072870001.txt
processing  SYP000072880001.txt
processing  SYP000072890001.txt
processing  SYP000072900001.txt
processing  SYP000072920001.txt
processing  SYP000072930001.txt
processing  SYP000072950001.txt
processing  SYP000072960001.txt
processing  SYP000072970001.txt
processing  SYP000072980001.txt
processing  SYP000072990001.txt
processing  SYP000073000001.txt
processing  SYP000073010001.txt
processing  SYP000073020001.txt
processing  SYP000073030001.txt
processing  SYP000073050001.txt
processing  SYP000073060001.txt
processing  SYP000073070001.txt
processing  SYP000073080001.txt
processing  SYP000073090001.txt
processing  SYP000073110001.txt
processing  SYP000073120001.txt
processing  SYP000073130001.txt
processing  SYP000073140001.txt
processing  SYP000073150001.txt
processing  SYP000073160001.txt
processing  SYP000073180001.txt
processi

KeyboardInterrupt: 

##### merge document indices into a corpus index
tmt.index_cooccurrence.merge_cooccurrence_matrices_for_corpus(cr.content_directory)

In [10]:
tmt.index_cooccurrence.print_matrix(cr.content_directory)

cooccurrence_matrix_file  data_sets/hillsborough/txt/matrix_cooccurrence.hdf5
               annes       april    ...       yorkshire      yours
annes      13.598966   69.452841    ...       20.132038  15.450841
april      16.647205  499.506645    ...      144.136850  44.767785
asked       1.509196   75.035781    ...       72.042577  20.361365
august     14.049427   41.647803    ...       81.874464  20.161213
authority  14.854351  104.800732    ...      314.637482  53.621280
available   3.389884   55.322686    ...       73.011968  38.755847
because     1.471518   81.453269    ...      109.463031  25.113478
being       1.103638   87.139681    ...      173.464271  52.827673
chief      13.027432  104.160590    ...      954.748559  35.329447
consider   18.622392   32.222886    ...       48.568918  73.922254

[10 rows x 100 columns]


In [11]:
# query the co-occurrence value for a given pair of words
tmt.index_cooccurrence.query_cooccurance_matrix(cr.content_directory, "chief", "police")

2048.3336048269839

In [12]:
# get the most likely next word by co-occurrence value for given word
tmt.index_cooccurrence.most_likely_next(cr.content_directory, "chief")

'constable'

In [18]:
#generate sentences based on co-occurence
sentence_length = 7

#initial words 
#for w in ["the", "once", "then", "olive", "tomato", "cut", "cook", "little", "boiled"]:
for w in ["police"]:
    sentence = w
    for i in range(sentence_length):
        w = tmt.index_cooccurrence.most_likely_next(cr.content_directory, w)
        sentence += (" " + w)
        pass
    print(sentence)
    pass


police police police police police police police police


In [19]:
words_by_co_occurrence = tmt.index_cooccurrence.get_word_pairs_by_cooccurrence(cr.content_directory)

In [20]:
# 20 most common
words_by_co_occurrence[:20]

,word1,word2,weight
0,there,there,1.000000
1,would,would,0.974656
2,police,police,0.866106
3,south,yorkshire,0.681754
4,should,should,0.671855
5,football,football,0.666573
6,which,which,0.566681
7,justice,taylor,0.520375
8,police,officers,0.503252
9,there,which,0.494395


In [21]:
# 20 least common
words_by_co_occurrence[-20:]

,word1,word2,weight
9957,sincerely,grateful,0.000070
9958,correspondence,turnstiles,0.000070
9959,paragraph,annes,0.000070
9960,officers,annes,0.000070
9961,inquest,annes,0.000058
9962,stuartsmith,division,0.000058
9963,annes,stand,0.000058
9964,whalley,these,0.000058
9965,annes,where,0.000055
9966,division,statements,0.000055


In [22]:
# visualise co-occurrence of words
tmt.visualisation.plot_force_directed_graph(words_by_co_occurrence[:500])

force-directed graph

In [24]:
# visualise co-occurrence > 1
w = words_by_co_occurrence[words_by_co_occurrence['weight']>0.2]
tmt.visualisation.plot_force_directed_graph(w)

force-directed graph